In [11]:
from langgraph.graph import StateGraph, START, END
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict
import os

In [12]:
load_dotenv()

True

In [13]:
# Get the key from env variable
groq_key = os.getenv("GROQ_API_KEY")

In [14]:
print("Loaded key:", groq_key is not None)  # Debug: should print Tru

Loaded key: True


In [15]:
model = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key
)

In [16]:
# create a state 

class LLMState(TypedDict):

    question:str
    answer:str

In [27]:
# create a function to ask the llm 
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from the state 
    question = state["question"]

    # form a prompt
    prompt = f"Answer the following question: {question}"

    # Ask that question to the LLM
    response = model.chat.completions.create(
        model="llama3-70b-8192",  # Use the Groq-supported model
        messages=[{"role": "user", "content": prompt}]
    )

    # Extract the answer
    answer = response.choices[0].message.content

    # update the state with the answer 
    state["answer"] = answer

    return state

In [28]:
# create our graph 

graph = StateGraph(LLMState)

# create our nodes 
graph.add_node("llm_qa", llm_qa)

#create our edges 
graph.add_edge(START, "llm_qa")
graph.add_edge("llm_qa", END)

#compile
workflow = graph.compile()


In [29]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state['answer'])

The average distance from the Earth to the Moon is about 384,400 kilometers (238,900 miles). This is called the "lunar distance" or "lunar mean distance." However, it's worth noting that the Moon's orbit is not a perfect circle and its distance from Earth varies slightly over the course of a month.

At its closest point, called "perigee," the Moon is about 356,400 kilometers (221,500 miles) away from Earth.

At its farthest point, called "apogee," the Moon is about 405,500 kilometers (252,000 miles) away from Earth.

So, to summarize:

* Average distance: 384,400 km (238,900 miles)
* Closest distance (perigee): 356,400 km (221,500 miles)
* Farthest distance (apogee): 405,500 km (252,000 miles)
